In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt  #이미지 확인용 모듈
import matplotlib.image as mpimg  #이미지 확인용 모듈
import os
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten # layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator  #데이터 확장 모듈

In [ ]:
train_dir = './train'
test_dir='./test'

In [ ]:
# train set의 데이터 확장 설정
# validation 20%로 나눔

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',   # data 확장시 어떤 방식을 취할지에 대한 파라미터.
    validation_split=0.2
)

In [ ]:
# 모델의 파라미터
input_shape = (224, 224, 3)
num_classes = 3  # 분류하려는 클래스 개수
batch_size = 32  # batch _size

In [ ]:
## train set에 대한 데이터 불러오기 

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # use the training subset of the data
)

## validation set에 대한 데이터 불러오기 

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' 
)


# test set 데이터 확장(generator) 설정. (test_set 이니깐 rescale 파라미터만 사용함)
test_datagen = ImageDataGenerator(rescale=1./255)

# test_set 에 대한 데이터 불러옴.
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
# Sequential 모델 생성: 층(layer)을 순차적으로 쌓을 수 있는 모델
model = Sequential()
# 1번째 합성곱층(Convolutional layer)
# 필터 수: 32, 커널 크기: (3, 3), 활성화 함수: ReLU
# input_shape: 입력 이미지의 형태 (높이, 너비, 채널 수)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# 1번째 최대풀링층(MaxPooling layer)
# 풀링 크기: (2, 2), 공간적 차원을 줄여 특징 추출
model.add(MaxPooling2D((2, 2)))
# 2번째 합성곱층
# 필터 수: 64, 커널 크기: (3, 3), 활성화 함수: ReLU
model.add(Conv2D(64, (3, 3), activation='relu'))
# 2번째 최대풀링층
model.add(MaxPooling2D((2, 2)))
# 3번째 합성곱층
# 필터 수: 128, 커널 크기: (3, 3), 활성화 함수: ReLU
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
# 완전연결층(Dense layer)
# 뉴런 수: 128, 활성화 함수: ReLU
model.add(Dense(128, activation='relu'))

# 출력층(Output layer)
# 뉴런 수: num_classes (분류 클래스 수), 활성화 함수: softmax
# softmax는 클래스 확률을 반환
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
# 옵티마이저: Adam (효율적이고 널리 쓰이는 최적화 알고리즘)
# 손실 함수: categorical_crossentropy (다중 클래스 분류에 적합)
# 평가 지표: 정확도(accuracy)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# (model.fit에 callbakcs=[Earlystopping] 추가하면 더 좋은 정확도가 나올것이다.)
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),  #train의 step수
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

In [ ]:
# Evaluate the model on the test data generator
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.n // batch_size, verbose=1)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
file_list=os.listdir(train_dir+'/Covid')
selecet_file=mpimg.imread(train_dir+'/Covid/'+file_list[1])  # 이미지의 픽셀값을 numpy로 반환
plt.imshow(selecet_file)
plt.show()